In [ ]:
import pandas as pd
import numpy as np
import geopandas as gpd
import osmnx as ox
import torch
import torch.nn as nn
from sklearn.preprocessing import StandardScaler
from pyproj import Transformer
from shapely.geometry import Point
import json
import warnings
warnings.filterwarnings('ignore')

# Define NoisePredictor class (from notebook)
class NoisePredictor(nn.Module):
    def __init__(self, input_dim):
        super(NoisePredictor, self).__init__()
        self.fc1 = nn.Linear(input_dim, 25)
        self.fc2 = nn.Linear(25, 50)
        self.fc3 = nn.Linear(50, 1)

    def forward(self, x):
        x = torch.relu(self.fc1(x))
        x = torch.relu(self.fc2(x))
        return self.fc3(x)

# Load trained model
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = NoisePredictor(input_dim=7)
model.load_state_dict(torch.load('MLP_noise_predictor_model.pth'))
model.to(device)
model.eval()

# Load sheets and process data (your provided code)
file_path = 'data\\FINAL DATA.xlsx'  # Update to your actual path
df_leq = pd.read_excel(file_path, sheet_name='Noise Leq Data')
df_speed = pd.read_excel(file_path, sheet_name='SPEED')
df_pcu = pd.read_excel(file_path, sheet_name='PCU')
df_pcu_conv = pd.read_excel(file_path, sheet_name='PCU Conversion')  # For vehicle counts

# Modified data processing to extract vehicle categories like the paper
data = []
places = df_leq['Place'].dropna().unique()
hours = ['6-7AM', '7-8AM', '8-9AM', '9-10AM', '10-11AM', '11-12PM', '12-1PM', '1-2PM', '2-3PM', '3-4PM', '4-5PM', '5-6PM']

lanes_dict = {
    'Around Arya School': 2,  # Ngara Arya
    'Around Baba Dogo Rd': 2,
    'Around Junction Mall': 4,
    'Around Langata Hospital': 4,
    'Around MMU': 4,
    'BBS Eastleigh': 4,
    'Bee Centre': 2,
    'Close to Uhuru Park': 2,
    'Davis&Shirtliff Kangundo Rd': 2,
    'ICD Road': 4,
    'Imaara Mall': 4,
    'Jevanjee': 2,
    'Jogoo Road': 4,
    'Kangemi': 4,
    'Karen C School': 2,
    'Kawangware': 2,
    'KCB Utawala Eastern Bypass': 4,
    'KFC Embakasi': 4,
    'Kiambu Road': 2,
    'Kiambu Road 2': 2,
    'Kinoo': 2,
    'Langata Link Road': 4,
    'Likoni Road': 4,
    'Makongeni Shopping Centre Ruai': 2,
    'Ngong Road': 4,
    'Northern Bypass': 2,
    'Nyayo Langata': 4,
    'Ola Energy Waiyaki Way': 8,
    'Opp. KU Hospital': 2,
    'Quality Meat Packers': 2,
    'Raila Odinga Road Next to Total': 4,
    'Ruaka': 2,
    'Runda': 2,
    'Southern Bypass 1': 4,
    'Southern Bypass 2': 4,
    'Thika Road 1': 8,
    'Thika Road 2': 8,
    'Thika Road (Pangani)': 8,
    'Thome': 2,
    'Total Energies Outering': 8,
    'Winners Chapel (Likoni Road)': 4,
    'Junction Mall': 4,
    'Arya (Ngara)': 2,
    'Around Baba Dogo Road': 2
}

# Coordinates from Coordinates.xlsx (hardcoded from provided document)
coords_dict = {
    'UHURU PARK': (256896.83, 9857588.15),
    'IMAARA MALL': (264136.81, 9853108.54),
    'KIAMBU ROAD': (259705.6, 9864042.77),
    'LANGATA LINK ROAD': (253374.75, 9853775.07),
    'JEVANJEE': (257452.84, 9858326.03),
    'KFC EMBAKASI': (268248.56, 9854218.08),
    'RUNDA': (258096.36, 9865454.82),
    'RAILA ODINGA ROAD NEXT TO TOTAL': (256590.06, 9854730.41),
    'ARYA(NGARA)': (257947.14, 9858910.23),
    'KCB UTAWALA EASTERN BYPASS': (272667.35, 9857939.17),
    'RUAKA': (258094.93, 9867236.88),
    'NYAYO LANGATA': (257702.58, 9855591.69),
    'OLA ENERGY WAIYAKI WAY': (252935.73, 9860718.79),
    'MAKONGENI SHOPPING CENTRE RUAI': (280644.38, 9858744.19),
    'KIAMBU ROAD 2': (258775.95, 9866407.85),
    'LIKONI ROAD': (259340.9, 9856361.23),
    'KANGEMI': (249008.64, 9860131.61),
    'QUALITY MEAT PACKERS': (272942.89, 9861441.77),
    'THOME': (263693.17, 9866165.94),
    'WINNERS CHAPEL(LIKONI ROAD)': (260611.02, 9853750.71),
    'KINOO': (243719.36, 9861018.07),
    'DAVIS & SHIRTLIFF KANGUNDO ROAD': (268614.18, 9860885.34),
    'NORTHERN BYPASS': (266227.84, 9867581.2),
    'LANGATA HOSPITAL': (253993.82, 9853130.38),
    'SOUTHERN BYPASS 1': (243042.87, 9856654.09),
    'BEE CENTRE': (266390.05, 9858026.26),
    'OPP. KU HOSPITAL': (267772.27, 9869917.36),
    'MMU': (251958.55, 9846860.37),
    'SOUTHERN BYPASS 2': (244992.07, 9855764.76),
    'TOTAL ENERGIES OUTERING': (264131.37, 9859591.32),
    'NORTHERN BYPASS 2': (268760.56, 9871638.6),
    'KAREN C SCHOOL': (248613.98, 9851927.16),
    'NGONG ROAD': (248517.89, 9855522.13),
    'JOGOO ROAD': (263515.43, 9856549.12),
    'THIKA ROAD 1': (270432.25, 9869181.99),
    'JUNCTION MALL': (251145.89, 9856292.64),
    'KAWANGWARE': (248423.11, 9857765.12),
    'BBS MALL EASTLEIGH': (260390.54, 9858451.41),
    'THIKA ROAD 2': (264838.81, 9864569.18),
    'ICD ROAD': (261941.98, 9852338.53),
    'THIKA ROAD(PANGANI)': (259152.23, 9860048.07),
    'AROUND BABA DOGO ROAD': (263418.15, 9862017.95)
}

for i, place in enumerate(places):
    leq_row = df_leq.iloc[i, 2:].values  # Leq per hour
    speed_row = df_speed.iloc[i, 2:].values  # Speed per hour
    pcu_row = df_pcu.iloc[i, 2:].values  # PCU per hour
    
    # Get vehicle categories from PCU Conversion
    place_conv = df_pcu_conv[df_pcu_conv['Place'].str.strip().str.upper() == place.strip().upper()]
    if not place_conv.empty:
        conv_row = place_conv.iloc[0, 3:].values  # Start from 'Bicycle' column
        vehicle_cols = df_pcu_conv.columns[3:]  # Columns from 'Bicycle' onward
        
        # Extract vehicle counts (handle missing columns with defaults)
        motorcycles = conv_row[list(vehicle_cols).index('Motorcycles')] if 'Motorcycles' in vehicle_cols else 0
        light_cols = ['Private car', 'Pickup', 'SUV']  # Define light vehicle categories
        light_vehicles = sum(conv_row[list(vehicle_cols).index(col)] for col in light_cols if col in vehicle_cols)
        medium_cols = ['Buses', 'Light trucks']  # Define medium vehicle categories
        medium_vehicles = sum(conv_row[list(vehicle_cols).index(col)] for col in medium_cols if col in vehicle_cols)
        heavy_cols = ['Medium trucks', 'Heavy trucks']  # Define heavy vehicle categories
        heavy_vehicles = sum(conv_row[list(vehicle_cols).index(col)] for col in heavy_cols if col in vehicle_cols)
        
        # Convert NaN to 0
        motorcycles = 0 if pd.isna(motorcycles) else motorcycles
        light_vehicles = 0 if pd.isna(light_vehicles) else light_vehicles
        medium_vehicles = 0 if pd.isna(medium_vehicles) else medium_vehicles
        heavy_vehicles = 0 if pd.isna(heavy_vehicles) else heavy_vehicles
    else:
        motorcycles = light_vehicles = medium_vehicles = heavy_vehicles = 0
    
    lanes = lanes_dict.get(place, 2)
    
    # Get X, Y coordinates
    x_coord, y_coord = coords_dict.get(place.strip().upper(), (None, None))
    
    # Process each hour
    for h, hour in enumerate(hours):
        if pd.notna(leq_row[h]) and pd.notna(speed_row[h]) and pd.notna(pcu_row[h]):
            current_speed = speed_row[h]
            
            # Flow type based on speed (like paper's traffic condition)
            if current_speed < 20:
                flow_type = 0  # Congested
            elif current_speed < 35:
                flow_type = 1  # Periodic
            else:
                flow_type = 2  # Fluid
            
            data.append({
                'place': place,
                'motorcycles': motorcycles,
                'light': light_vehicles,
                'medium': medium_vehicles,
                'heavy': heavy_vehicles,
                'speed': current_speed,
                'lanes': lanes,
                'flow_type': flow_type,
                'leq': leq_row[h],
                'hour': hours[h],
                'x': x_coord,
                'y': y_coord
            })

df = pd.DataFrame(data)
df = df.dropna(subset=['x', 'y'])  # Drop rows without coordinates

# Convert X, Y (UTM Zone 37S) to latitude, longitude (EPSG:4326)
transformer = Transformer.from_crs("EPSG:32737", "EPSG:4326", always_xy=True)  # UTM Zone 37S to WGS84
df['longitude'], df['latitude'] = zip(*df.apply(lambda row: transformer.transform(row['x'], row['y']), axis=1))

# Predict Leq
features = ['motorcycles', 'light', 'medium', 'heavy', 'speed', 'lanes', 'flow_type']
scaler = StandardScaler()
X = scaler.fit_transform(df[features].values)
X_tensor = torch.tensor(X, dtype=torch.float32).to(device)
with torch.no_grad():
    leq_preds = model(X_tensor).cpu().numpy().flatten()
df['leq_pred'] = leq_preds

# Get Nairobi road network
G = ox.graph_from_place('Nairobi, Kenya', network_type='drive')
gdf_nodes, gdf_edges = ox.graph_to_gdfs(G)

# Assign Leq to roads by nearest noise point
def get_nearest_noise_level(road_point, noise_gdf):
    distances = noise_gdf.geometry.distance(road_point)
    nearest_idx = distances.idxmin()
    return noise_gdf.loc[nearest_idx, 'leq_pred']
gdf_noise = gpd.GeoDataFrame(df, geometry=[Point(lon, lat) for lat, lon in zip(df['longitude'], df['latitude'])], crs="EPSG:4326")
gdf_edges['leq'] = gdf_edges.geometry.apply(lambda x: get_nearest_noise_level(Point(x.coords[0]), gdf_noise))

# Convert to GeoJSON
gdf_edges = gdf_edges[['geometry', 'leq', 'name']].dropna(subset=['leq'])
geojson_data = gdf_edges.to_geojson()

# Save GeoJSON
with open('nairobi_noise_data.geojson', 'w') as f:
    json.dump(json.loads(geojson_data), f)

print("GeoJSON data saved as 'nairobi_noise_data.geojson'")


AttributeError: 'GeoDataFrame' object has no attribute 'to_geojson'

: 